In [3]:
import re
import os

def parse_file(input_file, ctrl_to_sec):
    lines = []

    with open(input_file, 'r') as f:
        for line in f:
            line = line.strip()
            
            if line and not line.startswith('#'):
                parts = line.split('.')
                if len(parts) == 3:
                    x = int(parts[0][1:].split('Y')[0])
                    strings1, strings2 = adjust(x, parts[1], parts[2], ctrl_to_sec)
                    for i in range(len(strings1)):
                        lines.append(parts[0] + "." + strings1[i] + "." + strings2[i])
                else:
                    print("ERROR! not matching number of parts! " + line )

            else:
                lines.append(line)

    with open((input_file[:-5] + "__resolved.fasm"), 'w') as file:
        for line in lines:
            file.write(line + '\n')

    return lines

# string1 -> string2
def adjust(x, string1, string2, ctrl_to_sec):
    pattern = r'(.+?)(\d+|0_t|0_f|1_t|1_f)?$'
    strings1 = []
    strings2 = []

    new_indexes_SEC = {
        "0":["0","1","2","3"],
        "1":["4","5","6","7"],
        "2":["8","9","10","11"],
        "0_t":["0_t","0_f","1_t","1_f"]}

    new_indexes_CTRL = {
        "0":["0","1"],
        "1":["2","3"],
        "2":["4","5"],
        "0_t":["0_t","0_f"]}        

    if string1 != "A" and string1 != "A_T" and string2 != "A_T":

        # sec part
        if x <= ctrl_to_sec:
            if string1 == "VCC0":
                strings1.append("VCC0")
                strings1.append("GND0")
                strings1.append("GND0")
                strings1.append("VCC0")

            elif string1 == "GND0":
                strings1.append("GND0")
                strings1.append("VCC0")
                strings1.append("GND0")
                strings1.append("VCC0")

            else:
                match = re.match(pattern, string1)

                if match:
                    name1 = match.group(1)
                    index1 = match.group(2) if match.group(2) else print("ERROR!(index) regexp for arg: " + string1)

                    if "_1s" in name1:
                        strings1.append("GND0")
                        strings1.append("VCC0")
                        for i in range(2):
                            tmp_str = name1 + new_indexes_CTRL[str(index1)][i]
                            strings1.append(tmp_str)  

                    else:
                        for i in range(4):
                            tmp_str = name1 + new_indexes_SEC[str(index1)][i]
                            strings1.append(tmp_str)
                        
                else:
                    print("ERROR! regexp for arg: " + string1)
            
            match = re.match(pattern, string2)
            if match:
                name2 = match.group(1)
                index2 = match.group(2) if match.group(2) else print("ERROR!(index) regexp for arg: " + string2)                
                for i in range(4):
                    tmp_str = name2 + new_indexes_SEC[str(index2)][i]
                    strings2.append(tmp_str)                    
            else:
                print("ERROR! regexp for arg: " + string2)           

        # ctrl part
        else:
            if string1 == "VCC0":
                strings1.append("VCC0")
                strings1.append("GND0")

            elif string1 == "GND0":
                strings1.append("GND0")
                strings1.append("VCC0")

            else:
                match = re.match(pattern, string1)

                if match:
                    name1 = match.group(1)
                    index1 = match.group(2) if match.group(2) else print("ERROR!(index) regexp for arg: " + string1)

                    for i in range(2):
                        tmp_str = name1 + new_indexes_CTRL[str(index1)][i]
                        strings1.append(tmp_str)   

                        
                else:
                    print("ERROR! regexp for arg: " + string1)

            match = re.match(pattern, string2)
            if match:
                name2 = match.group(1)
                index2 = match.group(2) if match.group(2) else print("ERROR!(index) regexp for arg: " + string2)

                for i in range(2):
                    tmp_str = name2 + new_indexes_CTRL[str(index2)][i]
                    strings2.append(tmp_str)   
               
            else:
                print("ERROR! regexp for arg: " + string2)
    else:
        strings1.append(string1)
        strings2.append(string2)

    return strings1, strings2

In [8]:

main =  "/home/sergej/FABulous/FABulousFPGA/FABulous/23x7_SAUBER"
model = "/home/sergej/FABulous/FABulousFPGA/FABulous/23x7_SAUBER_model"
design = "tmp/AGEMA_design_2_fatwire_1bit"

input_file = model + "/user_design/" + design + "_des.fasm"

# 60x77 => 78 / 60x34 => 35 / 23x7 => 8
ctrl_to_sec_column = 8

parse_file(input_file,ctrl_to_sec_column)

my_cmd = "python3 /home/sergej/FABulous/FABulousFPGA/FABulous/fabric_cad/bit_gen.py -genBitstream " + model + "/user_design/" + design + "_des__resolved.fasm " + main + "/.FABulous/bitStreamSpec.bin " + main + "/AGEMA_Tests/test_design/" + design + ".bin"

os.system(my_cmd)



/home/sergej/FABulous/FABulousFPGA/FABulous/fabric_cad/bit_gen.py:72: SyntaxWarning: invalid escape sequence '\d'
  coordsRE = re.compile("X(\d*)Y(\d*)")
/home/sergej/FABulous/FABulousFPGA/FABulous/fabric_cad/bit_gen.py:225: SyntaxWarning: invalid escape sequence '\S'
  flagRE = re.compile("-\S*")
/home/sergej/anaconda3/lib/python3.12/site-packages/fasm/parser/__init__.py:30: RuntimeWarning: Unable to import fast Antlr4 parser implementation.
  ImportError: cannot import name 'antlr_to_tuple' from partially initialized module 'fasm.parser' (most likely due to a circular import) (/home/sergej/anaconda3/lib/python3.12/site-packages/fasm/parser/__init__.py)

  Falling back to the much slower pure Python textX based parser
  implementation.

  Getting the faster antlr parser can normally be done by installing the
  required dependencies and then reinstalling the fasm package with:
    pip uninstall
    pip install -v fasm

  warn(


0